<a href="https://colab.research.google.com/github/IsaiGowthami/CodeClause_Price_Recommendation_for_Online_Sellers/blob/main/Price_Recommendation_for_Online_Sellers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [46]:
import numpy as np
import pandas as pd


Reading and verifying the dataset in the input

In [47]:
df=pd.read_csv('/content/price_data.csv',encoding='latin1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


Collecting the info of input dataset

In [48]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112803 entries, 0 to 112802
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    112803 non-null  object 
 1   StockCode    112803 non-null  object 
 2   Description  112478 non-null  object 
 3   Quantity     112803 non-null  int64  
 4   InvoiceDate  112803 non-null  object 
 5   UnitPrice    112803 non-null  float64
 6   CustomerID   74868 non-null   float64
 7   Country      112802 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 6.9+ MB


describing the dataset

In [49]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,112803.000000,112803.000000,74868.000000
mean,8.864090,5.259118,15335.481995
std,317.034745,113.928561,1757.406625
min,-74215.000000,0.000000,12346.000000
25%,1.000000,1.250000,13875.000000
50%,3.000000,2.460000,15304.000000
75%,8.000000,4.250000,16951.000000
max,74215.000000,16888.020000,18283.000000


In [51]:
df.describe(include=object)

,InvoiceNo,StockCode,Description,InvoiceDate,Country
count,112803,112803,112478,112803,112802
unique,5310,3199,3108,4803,31
top,537434,85123A,WHITE HANGING HEART T-LIGHT HOLDER,12/6/2010 16:57,United Kingdom
freq,675,607,617,675,104115


Evalating and treating the missing value from dataset for efficent output

In [ ]:
df.isnull().sum()


InvoiceNo          0
StockCode          0
Description      299
Quantity           0
InvoiceDate        0
UnitPrice          1
CustomerID     34935
Country            1
dtype: int64

In [52]:
df = df.loc[df['Quantity'] > 0]
df = df.loc[df['UnitPrice'] > 0]

In [54]:
df.isnull().sum()

InvoiceNo          0
StockCode          0
Description        0
Quantity           0
InvoiceDate        0
UnitPrice          0
CustomerID     37273
Country            1
dtype: int64

In [55]:
df.loc[df['CustomerID'].isna()].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,12/1/2010 14:32,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,12/1/2010 14:32,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,12/1/2010 14:32,1.66,NaN,United Kingdom
1447,536544,21790,VINTAGE SNAP CARDS,9,12/1/2010 14:32,1.66,NaN,United Kingdom


In [56]:
df.nunique()


InvoiceNo      4042
StockCode      3122
Description    3070
Quantity        195
InvoiceDate    3798
UnitPrice       528
CustomerID     1790
Country          31
dtype: int64

In [57]:
df.shape

(110162, 8)

In [58]:
df = df.dropna(subset=['CustomerID'])

In [59]:
df.shape

(72889, 8)

In [60]:
df.isnull().sum()


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        1
dtype: int64

# **COLLABORATIVE FILTERING**

The models created by collaborative filtering techniques are based on the prior actions of a user (things previously chosen or purchased, and/or numerical ratings given to those items), as well as comparable choices made by other users. Then, this model is used to forecast the ratings for things or items themselves that the user could be interested in.

In [61]:
customer_item_matrix = df.pivot_table(index='CustomerID', columns='StockCode', values='Quantity',aggfunc='sum')
customer_item_matrix.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214K,90214M,90214P,90214R,90214S,90214V,BANK CHARGES,C2,M,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12347.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12348.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
12350.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
12352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,1.0


In [62]:
customer_item_matrix = customer_item_matrix.applymap(lambda x: 1 if x > 0 else 0)
customer_item_matrix.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214K,90214M,90214P,90214R,90214S,90214V,BANK CHARGES,C2,M,POST
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12347.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12348.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12350.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
12352.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [63]:
customer_item_matrix.shape


(1790, 2809)

# (A) Creating User-to-User Similarity Matrix

In [64]:
from sklearn.metrics.pairwise import cosine_similarity

user_user_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix))
user_user_sim_matrix

,0,1,2,3,4,5,6,7,8,9,...,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789
0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,1.000000,0.067344,0.069296,0.031944,0.095238,0.029627,0.000000,0.000000,0.030457,...,0.0,0.020620,0.038881,0.093522,0.000000,0.000000,0.070457,0.053995,0.090351,0.070041
2,0.0,0.067344,1.000000,0.057166,0.052705,0.196419,0.024441,0.074536,0.045361,0.000000,...,0.0,0.000000,0.000000,0.051434,0.000000,0.000000,0.154997,0.000000,0.000000,0.023113
3,0.0,0.069296,0.057166,1.000000,0.054233,0.040423,0.000000,0.076696,0.093352,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.039873,0.000000,0.000000,0.047565
4,0.0,0.031944,0.052705,0.054233,1.000000,0.149071,0.139122,0.141421,0.086066,0.095346,...,0.0,0.032275,0.000000,0.146385,0.000000,0.000000,0.036761,0.000000,0.000000,0.065779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1785,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.142857,0.000000,0.000000
1786,0.0,0.070457,0.154997,0.039873,0.036761,0.164399,0.051142,0.000000,0.000000,0.105150,...,0.0,0.047458,0.067116,0.107624,0.000000,0.000000,1.000000,0.000000,0.000000,0.096724
1787,0.0,0.053995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,1.000000,0.000000,0.037062
1788,0.0,0.090351,0.000000,0.000000,0.000000,0.000000,0.032791,0.000000,0.000000,0.000000,...,0.0,0.000000,0.043033,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000


In [65]:
user_user_sim_matrix.shape

(1790, 1790)

In [66]:
user_user_sim_matrix.columns = customer_item_matrix.index

user_user_sim_matrix['CustomerID'] = customer_item_matrix.index

user_user_sim_matrix = user_user_sim_matrix.set_index('CustomerID')
user_user_sim_matrix.head()

CustomerID,12346.0,12347.0,12348.0,12350.0,12352.0,12356.0,12359.0,12361.0,12362.0,12365.0,...,18233.0,18239.0,18245.0,18250.0,18257.0,18259.0,18260.0,18269.0,18280.0,18283.0
CustomerID,,,,,,,,,,,,,,,,,,,,,
12346.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
12347.0,0.0,1.000000,0.067344,0.069296,0.031944,0.095238,0.029627,0.000000,0.000000,0.030457,...,0.0,0.020620,0.038881,0.093522,0.0,0.0,0.070457,0.053995,0.090351,0.070041
12348.0,0.0,0.067344,1.000000,0.057166,0.052705,0.196419,0.024441,0.074536,0.045361,0.000000,...,0.0,0.000000,0.000000,0.051434,0.0,0.0,0.154997,0.000000,0.000000,0.023113
12350.0,0.0,0.069296,0.057166,1.000000,0.054233,0.040423,0.000000,0.076696,0.093352,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.039873,0.000000,0.000000,0.047565
12352.0,0.0,0.031944,0.052705,0.054233,1.000000,0.149071,0.139122,0.141421,0.086066,0.095346,...,0.0,0.032275,0.000000,0.146385,0.0,0.0,0.036761,0.000000,0.000000,0.065779


In [67]:
user_user_sim_matrix.loc[12557].sort_values(ascending=False)

CustomerID
12557.0    1.000000
15970.0    0.703526
12664.0    0.596285
14006.0    0.503953
17179.0    0.447214
             ...   
14533.0    0.000000
14532.0    0.000000
14530.0    0.000000
14525.0    0.000000
15298.0    0.000000
Name: 12557.0, Length: 1790, dtype: float64

In [68]:
items_bought_by_12557 = set(customer_item_matrix.loc[12557].iloc[customer_item_matrix.loc[12557].to_numpy().nonzero()].index)
items_bought_by_12557

{'20725',
 '20727',
 '20728',
 '22383',
 '22384',
 '84997A',
 '84997B',
 '84997C',
 '84997D'}

In [69]:
items_bought_by_12431 = set(customer_item_matrix.loc[12431.0].iloc[customer_item_matrix.loc[12431.0].to_numpy().nonzero()].index)
items_bought_by_12431

{'20675',
 '20677',
 '20685',
 '20724',
 '20725',
 '21000',
 '21001',
 '21080',
 '21094',
 '21115',
 '21217',
 '21239',
 '21240',
 '21242',
 '21243',
 '21244',
 '21245',
 '21428',
 '21524',
 '21527',
 '21533',
 '21537',
 '21555',
 '21622',
 '21731',
 '21791',
 '21933',
 '21936',
 '21937',
 '22131',
 '22191',
 '22192',
 '22193',
 '22194',
 '22195',
 '22196',
 '22204',
 '22219',
 '22326',
 '22328',
 '22329',
 '22352',
 '22354',
 '22356',
 '22382',
 '22466',
 '22467',
 '22505',
 '22629',
 '22631',
 '22726',
 '22727',
 '22728',
 '22729',
 '22730',
 '22785',
 '22786',
 '22846',
 '22941',
 '22951',
 '22971',
 '22975',
 '22976',
 '22977',
 '22988',
 '22992',
 '35004C',
 '35004G',
 '47504K',
 '48138',
 '79067',
 '84988',
 '85014A',
 '85014B',
 '85099B',
 '85184C',
 '85232A',
 '85232D',
 'POST'}

In [70]:
items_to_recommend_to_12557 = items_bought_by_12557 - items_bought_by_12431
items_to_recommend_to_12557

{'20727', '20728', '22383', '22384', '84997A', '84997B', '84997C', '84997D'}

In [72]:
df.loc[df['StockCode'].isin(items_to_recommend_to_12557), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode')

,Description
StockCode,
84997B,RED 3 PIECE RETROSPOT CUTLERY SET
84997C,BLUE 3 PIECE POLKADOT CUTLERY SET
22384,LUNCH BAG PINK POLKADOT
22383,LUNCH BAG SUKI DESIGN
20728,LUNCH BAG CARS BLUE
20727,LUNCH BAG BLACK SKULL.
84997D,PINK 3 PIECE POLKADOT CUTLERY SET
84997A,GREEN 3 PIECE POLKADOT CUTLERY SET
84997C,CHILDRENS CUTLERY POLKADOT BLUE


In [73]:
most_similar_user = user_user_sim_matrix.loc[12557].sort_values(ascending=False).reset_index().iloc[1, 0]
most_similar_user

15970.0

In [74]:
def get_items_to_recommend_cust(cust_a): 
  most_similar_user = user_user_sim_matrix.loc[cust_a].sort_values(ascending=False).reset_index().iloc[1, 0]
  items_bought_by_cust_a = set(customer_item_matrix.loc[cust_a].iloc[customer_item_matrix.loc[cust_a].to_numpy().nonzero()].index)
  items_bought_by_cust_b = set(customer_item_matrix.loc[most_similar_user].iloc[customer_item_matrix.loc[most_similar_user].to_numpy().nonzero()].index)
  items_to_recommend_to_a = items_bought_by_cust_b - items_bought_by_cust_a
  items_description = df.loc[df['StockCode'].isin(items_to_recommend_to_a), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode')
  return items_description

In [75]:
get_items_to_recommend_cust(12557.0)

,Description
StockCode,
22662,LUNCH BAG DOLLY GIRL DESIGN
22382,LUNCH BAG SPACEBOY DESIGN
22551,PLASTERS IN TIN SPACEBOY
85032C,CURIOUS IMAGES GIFT WRAP SET


In [76]:
get_items_to_recommend_cust(12431.0)


,Description
StockCode,
21212,PACK OF 72 RETROSPOT CAKE CASES
21238,RED RETROSPOT CUP
20676,RED RETROSPOT BOWL


# **(B) Creating Ite to Item similarity matrix**

In [77]:
item_item_sim_matrix = pd.DataFrame(cosine_similarity(customer_item_matrix.T))
item_item_sim_matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,2799,2800,2801,2802,2803,2804,2805,2806,2807,2808
0,1.000000,0.0,0.176777,0.117851,0.0,0.0,0.041667,0.034752,0.104116,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062994,0.048622,0.087706
1,0.000000,1.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,0.176777,0.0,1.000000,0.250000,0.0,0.0,0.000000,0.073721,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.051571,0.000000
3,0.117851,0.0,0.250000,1.000000,0.0,0.0,0.176777,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,0.000000,0.0,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [78]:
item_item_sim_matrix.shape


(2809, 2809)

In [79]:
item_item_sim_matrix.columns = customer_item_matrix.T.index

item_item_sim_matrix['StockCode'] = customer_item_matrix.T.index
item_item_sim_matrix = item_item_sim_matrix.set_index('StockCode')
item_item_sim_matrix.head()

StockCode,10002,10080,10120,10123C,10124A,10124G,10125,10133,10135,11001,...,90214K,90214M,90214P,90214R,90214S,90214V,BANK CHARGES,C2,M,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.0,0.176777,0.117851,0.0,0.0,0.041667,0.034752,0.104116,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062994,0.048622,0.087706
10080,0.000000,1.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
10120,0.176777,0.0,1.000000,0.250000,0.0,0.0,0.000000,0.073721,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.051571,0.000000
10123C,0.117851,0.0,0.250000,1.000000,0.0,0.0,0.176777,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000
10124A,0.000000,0.0,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000


In [80]:
item_item_sim_matrix.loc['10002'].sort_values(ascending=False)


StockCode
10002    1.000000
21826    0.222718
22554    0.200033
22944    0.192450
22407    0.192450
           ...   
22712    0.000000
22711    0.000000
22709    0.000000
22706    0.000000
22609    0.000000
Name: 10002, Length: 2809, dtype: float64

In [81]:
top_10_similar_items = list(item_item_sim_matrix.loc['10002'].sort_values(ascending=False).iloc[:10].index)
top_10_similar_items

['10002',
 '21826',
 '22554',
 '22944',
 '22407',
 '21635',
 '22973',
 '21731',
 '22537',
 '22620']

In [82]:
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [83]:
df.loc[df['StockCode'] == '90210A']


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
28848,538661,90210A,GREY ACRYLIC FACETED BANGLE,12,12/13/2010 15:42,1.25,15194.0,United Kingdom
28887,538662,90210A,GREY ACRYLIC FACETED BANGLE,12,12/13/2010 15:44,1.25,15159.0,United Kingdom
56707,541110,90210A,GREY ACRYLIC FACETED BANGLE,2,1/13/2011 15:11,2.95,15916.0,United Kingdom


In [84]:
df.loc[df['StockCode'] == '90210A'][:1]


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
28848,538661,90210A,GREY ACRYLIC FACETED BANGLE,12,12/13/2010 15:42,1.25,15194.0,United Kingdom


In [85]:
df.loc[df['StockCode'].isin(top_10_similar_items), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode').loc[top_10_similar_items]

,Description
StockCode,
10002,INFLATABLE POLITICAL GLOBE
21826,EIGHT PIECE DINOSAUR SET
22554,PLASTERS IN TIN WOODLAND ANIMALS
22944,CHRISTMAS METAL POSTCARD WITH BELLS
22407,MONEY BOX FIRST ADE DESIGN
21635,MADRAS NOTEBOOK LARGE
22973,CHILDREN'S CIRCUS PARADE MUG
21731,RED TOADSTOOL LED NIGHT LIGHT
22537,MAGIC DRAWING SLATE DINOSAUR


In [89]:
def get_top_similar_items(item):
  top_10_similar_items = list(item_item_sim_matrix.loc[item].sort_values(ascending=False).iloc[:10].index)
  top_10 = df.loc[df['StockCode'].isin(top_10_similar_items), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode').loc[top_10_similar_items]
  return top_10

In [90]:
get_top_similar_items('84029E')


,Description
StockCode,
84029E,RED WOOLLY HOTTIE WHITE HEART.
84029G,KNITTED UNION FLAG HOT WATER BOTTLE
21479,WHITE SKULL HOT WATER BOTTLE
22111,SCOTTIE DOG HOT WATER BOTTLE
21485,RETROSPOT HEART HOT WATER BOTTLE
22112,CHOCOLATE HOT WATER BOTTLE
22837,HOT WATER BOTTLE BABUSHKA
22837,HOT WATER BOTTLE BABUSHKA LARGE
22114,HOT WATER BOTTLE TEA AND SYMPATHY
